In [3]:
from chessdotcom import get_player_game_archives, get_leaderboards
from queue import Queue
import requests
import json

## Retrieve Games within a Certain ELO Range

In [99]:
def reformat(game):
    '''
    Signifcantly condenses the game game
    game: the dictionary of game
    '''
    components = game["pgn"].split("\n")[4:]
    indices=[2,4,7,8,11,13,14,15,16,17,19]  # Manually found
 
    for i in sorted(indices, reverse=True):
        del components[i]
    
    return components

def get_data(player, elo_lb, elo_ub, num_players_cap, num_games_cap):
    '''
    performs a bfs to get game data from a list of users of a desired elo.
    player: the player origin of the bfs.
    elo_lb: lower bound.
    elo_ub: upper bound.
    num_players_cap: number of players we want to cap our requests to.
    num_games_cap: number of games per player we want to cap our exploration at.
    '''
    num_games = 0
    games_lst = [] # populate a list of games.


    # Initialize a Queue
    user_list = Queue()
    user_list.put(player)

    # Mark the start node as visited
    visited = {player}

    # Loop until the queue is empty
    while len(visited) < num_players_cap: # stop when we have requested num_players_cap players.

        # Dequeue a vertex from the queue
        curr_user = user_list.get()

        # Request player games from server.
        requested_player = get_player_game_archives(curr_user)
        
        this_playergames = requests.get(url=requested_player.json["archives"][-1]).json()["games"]

        # Keep track of how many games we've seen from this player
        games_from_curr = 0

        # Add all adjacent vertices to the queue
        for game in this_playergames: # loop through the first num_games_cap games
            
            # Make sure it's a 5 min blitz
            if game['time_control'] == '180':
                # Detect what color and elo the opponent is
                black_player = game['black']['username']
                
                if curr_user == black_player:
                    opponent_elo = game['white']['rating']
                    opponent_user = black_player
                else:
                    opponent_elo = game['black']['rating']
                    opponent_user = game['black']['username']

                if opponent_elo >= elo_lb and opponent_elo <= elo_ub and opponent_user not in visited and opponent_user != curr_user:
                    
                    num_games += 1
                    if num_games % 100 == 0:
                        print(num_games)
                    
                    # print("Adding Game: " + curr_user + " vs. " + opponent_user)
                    #condensed = reformat(game)
                    games_lst.append(game)
                    user_list.put(opponent_user)

                    # Exit loop once we achieve the number of games cap
                    games_from_curr += 1
                    if games_from_curr >= num_games_cap:
                        break
        
        # Add user to visited to prevent duplicates
        visited.add(curr_user)
    
    return (games_lst, visited)

In [105]:
sample_high_elo_players = get_data(player="colinsong1", elo_lb=1390, elo_ub=1610, num_players_cap=500, num_games_cap=2)

100
200
300
400
500
600
700
800
900
1000


In [83]:
def store_games(games, file_name):
    '''
    Stores game dictionaries in a text file, separated by new line characters.

    games: a list of dictionaries of games to be stored.
    file_name: the path to a valid .txt file to store.
    '''
    with open(file_name, 'w') as f:
        for game in games:
            # json.dumps converts games to a string for storage
            f.write(json.dumps(game))
            f.write("\n")

def read_games(file_name):
    '''
    Retrieves games stored in a text file.

    file_name: the path to a valid .txt file with stored games.
    '''
    games = []

    with open(file_name, 'r') as f:
        for line in f:
            games.append(json.loads(line))

    return games

In [106]:
print(len(sample_high_elo_players[0]))

1043


In [107]:
store_games(sample_high_elo_players[0], "games.txt")

games = read_games("games.txt")
games[0]

{'url': 'https://www.chess.com/game/live/74199703367',
 'pgn': '[Event "Live Chess"]\n[Site "Chess.com"]\n[Date "2023.04.02"]\n[Round "-"]\n[White "colinsong1"]\n[Black "TimeFish_576"]\n[Result "0-1"]\n[CurrentPosition "r1bq1rk1/ppp2pbp/2n3pB/8/8/1P1P4/P1Pp1PPP/2KR2NR w - -"]\n[Timezone "UTC"]\n[ECO "B06"]\n[ECOUrl "https://www.chess.com/openings/Modern-Defense-with-1-e4-2.Nc3-Bg7"]\n[UTCDate "2023.04.02"]\n[UTCTime "18:36:37"]\n[WhiteElo "1604"]\n[BlackElo "1609"]\n[TimeControl "180"]\n[Termination "TimeFish_576 won by resignation"]\n[StartTime "18:36:37"]\n[EndDate "2023.04.02"]\n[EndTime "18:37:04"]\n[Link "https://www.chess.com/game/live/74199703367"]\n\n1. e4 {[%clk 0:03:00]} 1... g6 {[%clk 0:02:59.9]} 2. Nc3 {[%clk 0:02:59]} 2... Bg7 {[%clk 0:02:59.6]} 3. Bc4 {[%clk 0:02:58.3]} 3... e6 {[%clk 0:02:59.3]} 4. d3 {[%clk 0:02:57.8]} 4... Ne7 {[%clk 0:02:59.1]} 5. Be3 {[%clk 0:02:57.2]} 5... O-O {[%clk 0:02:58.8]} 6. Qd2 {[%clk 0:02:56.7]} 6... Nbc6 {[%clk 0:02:58.5]} 7. O-O-O {[%clk 

Way to get games:
1) Find any 1500ish player
2) Get a random game
3) Go into their opponents account and get a random game
4) Repeat
5) Compile all of these games to make a set

Things to consider:
- Ratings are wrong when accounts are new or have low game numbers
- Pick games from unique players
- Python chess has opening and endgame databases
https://python-chess.readthedocs.io/en/latest/

Action Items:
- Parse PGN
- Find an efficient way to gather data
    - Make the dataset of PGNs
- Compute time differences between moves
- Make a "stage of game" variable to classify opening/mid/endgame
- EDA of times. Summary statistics of time vs stage of game, etc

Observations:
- Shows opening URL. Could help in game stage classification
- Current Board is a thing
